In [0]:
from pyspark.sql import SparkSession

# Criar sessão Spark (caso ainda não tenha sido criada)
spark = SparkSession.builder.appName("VisualizandoDataset").getOrCreate()

In [0]:
# Caminho do dataset
caminho_dataset = "dbfs:/FileStore/tables/entregas-2.csv"
# Visualizando o dataset 
df_entregas = spark.read.csv(caminho_dataset, header=True, inferSchema=True)
df_entregas.show()




+----------+-----------+------------+--------------+------------+--------+
|ID_Entrega|Data_Pedido|Data_Entrega|Transportadora|Distancia_km|  Status|
+----------+-----------+------------+--------------+------------+--------+
|         1| 2024-02-01|  2024-02-05|             A|         100|Entregue|
|         2| 2024-02-03|  2024-02-06|             B|         250|Atrasado|
|         3| 2024-02-02|  2024-02-04|             A|          50|Entregue|
|         4| 2024-02-05|  2024-02-10|             C|         500|Atrasado|
|         5| 2024-02-06|  2024-02-07|             B|          30|Entregue|
+----------+-----------+------------+--------------+------------+--------+



In [0]:
# Análise descritiva dos dados 
df_entregas.describe().show()

+-------+------------------+--------------+------------------+--------+
|summary|        ID_Entrega|Transportadora|      Distancia_km|  Status|
+-------+------------------+--------------+------------------+--------+
|  count|                 5|             5|                 5|       5|
|   mean|               3.0|          null|             186.0|    null|
| stddev|1.5811388300841898|          null|195.52493447128424|    null|
|    min|                 1|             A|                30|Atrasado|
|    max|                 5|             C|               500|Entregue|
+-------+------------------+--------------+------------------+--------+



In [0]:
from pyspark.sql.functions import col, count, when

# Contar o número total de entregas e atrasos 
total_entregas = df_entregas.count()
total_atrasos = df_entregas.filter(col("Status") == "Atrasado").count()

print(total_entregas)
print(total_atrasos)

5
2


In [0]:
# Calcular  taxa de atraso
taxa_atraso = (total_atrasos/total_entregas) * 100
print(f"Taxa de atraso: {taxa_atraso:.2f}%")

Taxa de atraso: 40.00%


In [0]:
# Analisar atrasos por transportadora
df_entregas.groupBy("Transportadora").agg(count(when(col("Status") == "Atrasado", True)).alias("Total_Atrasos")).show()

+--------------+-------------+
|Transportadora|Total_Atrasos|
+--------------+-------------+
|             B|            1|
|             C|            1|
|             A|            0|
+--------------+-------------+



In [0]:
# Verificar relação entre distância e atrasos 
df_entregas.groupBy("Distancia_km").agg(count(when(col("Status") == "Atrasado", True)).alias("Total_Atrasos")).show()

+------------+-------------+
|Distancia_km|Total_Atrasos|
+------------+-------------+
|         500|            1|
|         250|            1|
|         100|            0|
|          50|            0|
|          30|            0|
+------------+-------------+

